# 使用 CLI 发布适用于 Windows 的打包 .NET MAUI 应用

分发适用于 Windows 的 .NET 多平台应用 UI （.NET MAUI） 应用时，可以将应用及其依赖项发布到文件夹，以便部署到另一个系统。还可以将应用打包到 MSIX 包中，这对安装应用的用户具有许多好处。有关 MSIX 优势的详细信息，请参阅[什么是 MSIX](https://learn.microsoft.com/en-us/windows/msix/overview)？

## 创建签名证书
您必须使用签名证书才能用于发布应用。此证书用于对 MSIX 包进行签名。以下步骤演示如何使用 PowerShell 创建和安装自签名证书：The following steps demonstrates how to create and install a self-signed certificate with PowerShell：

1. 打开 PowerShell 终端并导航到包含项目的目录。
2. 使用 New-SelfSignedCertificate 命令生成自签名证书。

In [4]:
New-SelfSignedCertificate -Type Custom -Subject "CN=PublisherName" -KeyUsage DigitalSignature -FriendlyName "My temp dev cert" -CertStoreLocation "Cert:\CurrentUser\My" -TextExtension @("2.5.29.37={text}1.3.6.1.5.5.7.3.3", "2.5.29.19={text}")


   PSParentPath: Microsoft.PowerShell.Security\Certificate::CurrentUser\My

Thumbprint                                Subject              EnhancedKeyUsageList
----------                                -------              --------------------
8DBD2DC3182B2E8D34D69C10EE726162D26EED21  CN=PublisherName     代码签名



3. 使用以下 PowerShell 命令查询已创建的证书的证书存储：

In [5]:
Get-ChildItem "Cert:\CurrentUser\My" | Format-Table Thumbprint, Subject, FriendlyName


Thumbprint                               Subject                                       FriendlyName
----------                               -------                                       ------------
FE50D9D1B01D247833798CD5F892CCE98E3153DE CN=c09ecfdf-4b83-44e5-bef5-f21bf76bea27       Microsoft Y…
E7088E24C51B092161589B53A163B042DE1A94B7 CN=b9cb5587-0c8a-4657-9fab-b73bb1e04b83       CrossDevice
C6AA7AA6A3DCA1E020161A05117853234D3FCB0C CN=trust_                                     Microsoft Y…
9BD0407351D4B15EBA9F532867F08312DB6CCF30 CN=trust_b9cb5587-0c8a-4657-9fab-b73bb1e04b83 CrossDevice
6E73B7FEB7139A5608782EE9DA41481701B8CCDD CN=trust_c09ecfdf-4b83-44e5-bef5-f21bf76bea27 Microsoft Y…
4151579F4BAE853B16EB87E690EF5B70C151EA10 CN=3ac8518c-736b-4629-8146-3d203e6a27c2       Microsoft Y…
008AF8F5D3CC3491BC962048D02B0246B63F6A32 CN=localhost                                  ASP.NET Cor…
8DBD2DC3182B2E8D34D69C10EE726162D26EED21 CN=PublisherName                              My temp dev…



In [ ]:
#创建证书，修改<>内的内容。
New-SelfSignedCertificate -Type Custom -Subject "CN=<名字>, C=<国家>" -KeyUsage DigitalSignature -FriendlyName "<名字>" -CertStoreLocation "Cert:\CurrentUser\My" -TextExtension @("2.5.29.37={text}1.3.6.1.5.5.7.3.3", "2.5.29.19={text}") -NotAfter (Get-Date).AddYears(10)
#设置当前目录为 Cert:\CurrentUser\My。
Set-Location Cert:\CurrentUser\My
#设置密码，修改<Your Password>。
$password = ConvertTo-SecureString -String <Your Password> -Force -AsPlainText 
#导出 pfx 证书到 <FilePath>,<Thumbprint> 替换为之前查询到的 Thumbprint,用于给应用签名。
Export-PfxCertificate -cert "Cert:\CurrentUser\My\<Certificate Thumbprint>" -FilePath <FilePath>.pfx -Password $password
#导出 cer 证书 <FilePath>，用于分发应用。
Export-Certificate -cert "Cert:\CurrentUser\My\<Thumbprint>" -FilePath <FilePath>.cer

#将 pfx 证书序列化
certutil -encode <FilePath>.pfx <FilePath>.pfx.asc
#查看序列化后证书内容
cat <FilePath>.pfx.asc

WINDOWS_PFX_FILE  
WINDOWS_PFX_PASSWORD  
WINDOWS_PFX_THUMBPRINT